<a href="https://colab.research.google.com/github/jwang44/crispy-fiesta/blob/main/Bernoulli_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import numpy as np
import time
import scipy

In [135]:
class Bernoulli_NB():
  def __init__(self, LaplaceSmoothing = True):
    self.LaplaceSmoothing = LaplaceSmoothing
    self.Prob_Y = None      # P(Y)
    self.Prob_X_Y = None     # P(xj|Y)
    self.Prob_X_Y = None     # P(xj|Y)
    self.n_class = 0
    self.w0 = None
    self.w = None


  def ProbY(self, Y):
    # calculate P(Y=1) and P(Y=0)
    ProbY = np.zeros((1,2))
    #print('shape',np.shape(Y))
    ProbY[0,1] = np.sum(Y)/np.shape(Y)
    ProbY[0,0] = 1 - ProbY[0,1]
    return ProbY

  def ProbX_Yi(self, X, Y, label):
    # calculte P(xj=1|Yi=1)
    rows,cols = X.shape       # feature shape
    numerator = np.zeros(cols)    # initialize numerator

    # xj=1 and Yi=1
    for n in range(rows):
      if Y[n] == label:
        numerator += X[n,:]
    # Yi=1
    denominator = np.count_nonzero(Y == label)
    # Laplace Smoothing
    if(self.LaplaceSmoothing):
      numerator += 1
      denominator += 2
    # P(xj=1|Yi=1)
    prob = numerator/denominator
    return prob    

  def fit(self, X, Y):
    print('----------------------start fitting---------------------')
    t1 = time.time()

    rows,cols = X.shape              # feature shape
    self.n_class = len(np.unique(Y))       # number of classes
    self.Prob_Y = np.zeros((self.n_class,2))   # initialize P(Y)
    self.Prob_X_Y = np.zeros((self.n_class,2,cols)) # initialize P(x|Y)
    c = np.zeros((self.n_class,cols))       # rows:class cols:xj
    d = np.zeros((self.n_class,cols))       # rows:class cols:xj
    self.w0 = np.zeros((1,self.n_class))     # [w0Y1,w0Y2,...]
    self.w = np.zeros((self.n_class,cols))    # [(w1,w2,...)Y1;
                            # (w1,w2,...)Y2]

    for Yi in range(self.n_class):
      Y_onevsall = np.where(Y == Yi, 1, 0)    # only have 2 classes: Yi(1) & notYi(0)
      self.Prob_Y[Yi,:] = self.ProbY(Y_onevsall)   # [P(notYi), P(Yi)]
      self.Prob_X_Y[Yi,0,:] = self.ProbX_Yi(X,Y_onevsall,0)  # [P(x1|notYi), P(x2|notYi),...]
      self.Prob_X_Y[Yi,1,:] = self.ProbX_Yi(X,Y_onevsall,1)  # [P(x1|Yi), P(x2|Yi),...]
      c[Yi,:] = np.log10(self.Prob_X_Y[Yi,1,:]/self.Prob_X_Y[Yi,0,:])     # log(P(xj|Y=1)/P(xj|Y=0))
      d[Yi,:] = np.log10((1-self.Prob_X_Y[Yi,1,:])/(1-self.Prob_X_Y[Yi,0,:])) # log((1-P(xj|Y=1))/(1-P(xj|Y=0)))
      self.w0[0,Yi] = np.log10(self.Prob_Y[Yi,1]/self.Prob_Y[Yi,0]) + np.sum(d[Yi,:])
      self.w[Yi,:] = c[Yi,:] - d[Yi,:]

    print('------fit done, total time:',time.time()-t1,' -----')
    # return self.Prob_Y,self.Prob_X_Y

  def predict(self, X):
    print('----------------------start predict---------------------')
    t1 = time.time()

    rows,cols = X.shape       # feature shape
    PreY = np.zeros((rows,1),dtype=int) # initialize Y
    LogOddsRatio = np.zeros((1,self.n_class))  # initialize log odds ratio a(x) 
    Logistic = np.zeros((1,self.n_class))    # initialize logistic function
    for obs in range(rows):
      for Yi in range(self.n_class):
        LogOddsRatio[0,Yi] = self.w0[0,Yi] + np.sum((self.w[Yi,:]*X[obs,:])) 
        Logistic[0,Yi] = 1/(1+np.exp(-LogOddsRatio[0,Yi]))
      PreY[obs] = np.where(Logistic == np.amax(Logistic))[1]
    print('------predict done, total time:',time.time()-t1,' -----')
    return PreY

  def Accu_eval(self,PreY,TrueY):
    #y_predict = self.predict(X_test)
    rows,cols = np.shape(PreY)
    TP = 0;FP = 0;TN = 0;FN = 0
    # count TP,TN,FP,FN in validation set
    for obs in range(rows):
      if  TrueY[obs]==1 and PreY[obs]==1:
        TP = TP+1
      elif TrueY[obs]==0 and PreY[obs]==0:
        TN = TN+1
      elif TrueY[obs]==0 and PreY[obs]==1:
        FP = FP+1
      elif TrueY[obs]==1 and PreY[obs]==0:
        FN = FN+1    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F = 2*precision*recall/(precision+recall)
    specificity = TN/(FP+TN)
    FPR = FP/(FP+TN)
    print("---------------------accuracy:",accuracy,'-------------------')
    return accuracy


In [136]:
Y = np.array([0,1,1,1,2])
X = np.array([[0,1],
              [1,0],
              [1,1],
              [1,1],
              [0,0]])
B = Bernoulli_NB()
B.fit(X,Y)
Xtest = np.array([[0,1],
              [1,0],
              [0,0]])
Ytest = np.array([0,1,2])
PreY = B.predict(Xtest)
B.Accu_eval(PreY,Ytest)

----------------------start fitting---------------------
------fit done, total time: 0.0009458065032958984  -----
----------------------start predict---------------------
------predict done, total time: 0.00037026405334472656  -----
---------------------accuracy: 1.0 -------------------


1.0